In [43]:
!pip install kaggle -q
!pip install tensorflow-addons -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.7 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [3]:
!mkdir ~/.kaggle
!cp /content/gdrive/MyDrive/kaggle.json ~/.kaggle
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d williamscott701/memotion-dataset-7k

100% 694M/695M [00:31<00:00, 25.1MB/s]
100% 695M/695M [00:31<00:00, 23.0MB/s]


In [6]:
!unzip -qq /content/memotion-dataset-7k.zip

In [7]:
import re
import string
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPool2D, GlobalAveragePooling2D
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Activation, Dropout
from tensorflow.keras.layers import Conv1D, Embedding, GlobalAveragePooling1D 
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.preprocessing import image


In [8]:
df = pd.read_csv('/content/memotion_dataset_7k/labels.csv')
df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
df = df.drop(columns = ['text_ocr'])
df.head()

,image_name,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment
0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,hilarious,general,not_offensive,not_motivational,very_positive
1,image_2.jpeg,The best of #10 YearChallenge! Completed in le...,not_funny,general,not_offensive,motivational,very_positive
2,image_3.JPG,Sam Thorne @Strippin ( Follow Follow Saw every...,very_funny,not_sarcastic,not_offensive,not_motivational,positive
3,image_4.png,10 Year Challenge - Sweet Dee Edition,very_funny,twisted_meaning,very_offensive,motivational,positive
4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,hilarious,very_twisted,very_offensive,not_motivational,neutral


In [9]:
full_df = df.copy()
full_df.isnull().any()

image_name           False
text_corrected        True
humour               False
sarcasm              False
offensive            False
motivational         False
overall_sentiment    False
dtype: bool

In [10]:
clean_df = df.copy()
clean_df.dropna(inplace=True)
clean_df.isnull().any()

image_name           False
text_corrected       False
humour               False
sarcasm              False
offensive            False
motivational         False
overall_sentiment    False
dtype: bool

In [11]:
df = df.replace({'humour': {'not_funny': 0, 'funny': 1, 'very_funny': 2, 'hilarious':3},
            'sarcasm': {'not_sarcastic': 0, 'general': 1, 'twisted_meaning': 2, 'very_twisted': 3},
            'offensive': {'not_offensive': 0, 'slight': 1, 'very_offensive': 2, 'hateful_offensive': 3},
            'motivational': {'not_motivational': 0, 'motivational': 1},
            'overall_sentiment': {'very_negative': 0, 'negative': 1, 'neutral': 2, 'positive': 3, 'very_positive': 4}})

In [12]:
from PIL import ImageFile, ImageOps
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [13]:
width = 100
height = 100
X_full = []
X_full_path = []
for i in tqdm(range(full_df.shape[0])):
    path = '/content/memotion_dataset_7k/images/'+full_df['image_name'][i]
    img = image.load_img(path,target_size=(width,height,3))
    img = image.img_to_array(img)
    img = img/255.0
    X_full.append(img)
    X_full_path.append(path)

 93%|█████████▎| 6500/6992 [00:41<00:02, 168.97it/s]/usr/local/lib/python3.8/dist-packages/PIL/TiffImagePlugin.py:788: UserWarning: Corrupt EXIF data.  Expecting to read 2 bytes but only got 0. 
  warnings.warn(str(msg))
 95%|█████████▌| 6664/6992 [00:42<00:01, 185.28it/s]/usr/local/lib/python3.8/dist-packages/PIL/Image.py:959: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(
100%|██████████| 6992/6992 [00:44<00:00, 156.75it/s]


In [14]:
width = 100
height = 100
X_cleaned = []
X_cleaned_path = []
for i in tqdm(range(clean_df.shape[0])):
    if i in [119, 4799, 6781, 6784, 6786]:
        pass
    else:
        path = '/content/memotion_dataset_7k/images/'+clean_df['image_name'][i]
        img = image.load_img(path,target_size=(width,height,3))
        img = image.img_to_array(img)
        img = img/255.0
        X_cleaned.append(img)
        X_cleaned_path.append(path)

100%|██████████| 6987/6987 [00:44<00:00, 157.07it/s]


In [15]:
deleted_paths = [paths for paths in X_full_path + X_cleaned_path if paths not in X_full_path or paths not in X_cleaned_path]
deleted_paths

['/content/memotion_dataset_7k/images/image_120.jpg',
 '/content/memotion_dataset_7k/images/image_4800.jpg',
 '/content/memotion_dataset_7k/images/image_6782.jpg',
 '/content/memotion_dataset_7k/images/image_6785.jpg',
 '/content/memotion_dataset_7k/images/image_6787.jpg',
 '/content/memotion_dataset_7k/images/image_6988.jpg',
 '/content/memotion_dataset_7k/images/image_6989.jpg',
 '/content/memotion_dataset_7k/images/image_6990.png',
 '/content/memotion_dataset_7k/images/image_6991.jpg',
 '/content/memotion_dataset_7k/images/image_6992.jpg']

In [16]:
delete_var = [full_df, clean_df, X_cleaned, X_full, X_cleaned_path, X_full_path]
for i in delete_var:
    del i

In [17]:
width = 100
height = 100
X = []
for i in tqdm(range(clean_df.shape[0])):
    if i in [119, 4799, 6781, 6784, 6786]:
        pass
    else:
        path = '/content/memotion_dataset_7k/images/'+clean_df['image_name'][i]
        img = image.load_img(path,target_size=(width,height,3))
        img = image.img_to_array(img)
        img = img/255.0
        X.append(img)

100%|██████████| 6987/6987 [00:44<00:00, 156.70it/s]


In [18]:
X = np.array(X)

In [19]:
X.shape

(6982, 100, 100, 3)

In [20]:
rows_to_drop = ['image_120.jpg',
              'image_4800.jpg',
              'image_6782.jpg',
              'image_6785.jpg',
              'image_6787.jpg',
              'image_6988.jpg',
              'image_6989.jpg',
              'image_6990.png',
              'image_6991.jpg',
              'image_6992.jpg']

In [21]:
cleaner_df = df
cleaner_df.head()

,image_name,text_corrected,humour,sarcasm,offensive,motivational,overall_sentiment
0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,3,1,0,0,4
1,image_2.jpeg,The best of #10 YearChallenge! Completed in le...,0,1,0,1,4
2,image_3.JPG,Sam Thorne @Strippin ( Follow Follow Saw every...,2,0,0,0,3
3,image_4.png,10 Year Challenge - Sweet Dee Edition,2,2,2,1,3
4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,3,3,2,0,2


In [22]:
for images in rows_to_drop:
    cleaner_df.drop(cleaner_df[cleaner_df['image_name'] == images].index, inplace=True)

In [23]:
cleaner_df.shape

(6982, 7)

In [24]:
np.save('image_array', X)

In [25]:
Y = cleaner_df.iloc[:,2:]
Y.shape

(6982, 5)

In [27]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal'),
  tf.keras.layers.experimental.preprocessing.RandomRotation(0.2),
])

preprocess_input = tf.keras.applications.resnet_v2.preprocess_input

rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)

In [28]:
base_model_1 = tf.keras.applications.ResNet50(input_shape=X[0].shape,
                                               include_top=False,
                                               weights='imagenet')
base_model_2 = tf.keras.applications.VGG16(input_shape=X[0].shape,
                                               include_top=False,
                                               weights='imagenet')

58889256/58889256 [==============================] - 3s 0us/step


In [29]:
base_model_1.trainable = False
base_model_2.trainable = False

In [30]:
global_average_layer = GlobalAveragePooling2D()

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2)

In [33]:
del X

In [34]:
image_input = tf.keras.Input(shape=(100, 100, 3), name = 'image_input')
iamge_layers = data_augmentation(image_input)
iamge_layers = preprocess_input(iamge_layers)
layer_bm_1 = base_model_1(image_input, training=False)
layer_bm_1 = Conv2D(2048, kernel_size=2,padding='valid')(layer_bm_1)
layer_bm_1 = Dense(512)(layer_bm_1)
layer_bm_2 = base_model_2(image_input, training=False)
layer_bm_2 = Dense(512)(layer_bm_2)
layers = tf.keras.layers.concatenate([layer_bm_1, layer_bm_2])
iamge_layers = global_average_layer(layer_bm_1)
image_layers = Dropout(0.2, name = 'dropout_layer')(iamge_layers)

In [35]:
def standardization(data):
    data = data.apply(lambda x: x.lower())
    data = data.apply(lambda x: re.sub(r'\d+', '', x))
    data = data.apply(lambda x: re.sub(r'.com', '', x, flags=re.MULTILINE))
    data = data.apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
    return data

df['text_corrected'] = standardization(df.text_corrected)

In [36]:
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
vocab_size = 10000
sequence_length = 100

vectorize_layer = TextVectorization(
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

text_ds = np.asarray(df['text_corrected'])
vectorize_layer.adapt(tf.convert_to_tensor(text_ds))

In [37]:
X_text_train, X_text_test, y_text_train, y_text_test = train_test_split(cleaner_df.text_corrected, Y, test_size = 0.2)


In [38]:
embedding_dim=50

text_input = tf.keras.Input(shape=(1,), dtype=tf.string, name='text')
text_layers = vectorize_layer(text_input)
text_layers = tf.keras.layers.Embedding(vocab_size, embedding_dim, name="embedding")(text_layers)
text_layers = tf.keras.layers.Dropout(0.5)(text_layers)

text_layers = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, activation='relu', return_sequences=True))(text_layers)

text_layers = tf.keras.layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(text_layers)
text_layers = tf.keras.layers.Conv1D(128, 7, padding="valid", activation="relu", strides=3)(text_layers)
text_layers = tf.keras.layers.GlobalMaxPooling1D()(text_layers)

# We add a vanilla hidden layer:
text_layers = tf.keras.layers.Dense(2048, activation="relu")(text_layers)
text_layers = tf.keras.layers.Dropout(0.5)(text_layers)

In [39]:
concatenate = tf.keras.layers.concatenate([image_layers, text_layers], axis=1)

In [40]:
overall_layer = tf.keras.layers.Dense(2048, activation='softmax')(concatenate)

In [41]:
prediction_layer_1 = tf.keras.layers.Dense(4, activation='softmax', name = 'sarcasm')
prediction_layer_2 = tf.keras.layers.Dense(4, activation='softmax', name = 'humuor')
prediction_layer_3 = tf.keras.layers.Dense(4, activation='softmax', name = 'offensive')
prediction_layer_4 = tf.keras.layers.Dense(1, activation='sigmoid', name = 'motivational')
prediction_layer_5 = tf.keras.layers.Dense(5, activation='softmax', name = 'overall')

In [42]:
output_1 = prediction_layer_1(overall_layer)
output_2 = prediction_layer_2(overall_layer)
output_3 = prediction_layer_3(overall_layer)
output_4 = prediction_layer_4(overall_layer)
output_5 = prediction_layer_5(overall_layer)
model = tf.keras.Model(inputs = [image_input, text_input] , outputs = [output_1, output_2, output_3, output_4, output_5])

In [44]:
import tensorflow_addons as tfa

In [45]:
base_learning_rate = 0.01
losses = {
      "humuor": tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
      "sarcasm": tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
      "offensive": tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
      "motivational": tf.keras.losses.BinaryCrossentropy(from_logits=False),
      "overall": tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
}
lossWeights = {
      "humuor": 1.0, 
      "sarcasm": 1.0, 
      "offensive": 1.0, 
      "motivational": 1.0,
      "overall": 1.0
}
metric = {
    "humuor": ['acc',tfa.metrics.F1Score(num_classes=4, average="micro", threshold = 0.9)],
    "sarcasm": ['acc',tfa.metrics.F1Score(num_classes=4, average="micro", threshold = 0.9)],
    "offensive": ['acc',tfa.metrics.F1Score(num_classes=4, average="micro", threshold = 0.9)],
    "motivational": ['acc',tfa.metrics.F1Score(num_classes=1, average="micro", threshold = 0.9)],
    "overall": ['acc',tfa.metrics.F1Score(num_classes=5, average="micro", threshold = 0.9)]
}
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss = losses,
              loss_weights= lossWeights,
              metrics=metric)

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [46]:
learning_rates = [0.1] #, 0.01, 0.001, 0.0001, 0.00001]
for i in learning_rates:
    # model.compile(optimizer=tf.keras.optimizers.RMSprop(i),
    #           loss = losses,
    #           loss_weights= lossWeights,
    #           metrics=['accuracy'])
    
    history = model.fit(x = {"image_input": X_train, "text": X_text_train},
                    y = {"sarcasm": y_train.sarcasm, 
                         "humuor": y_train.humour, 
                         "offensive": y_train.offensive, 
                         "motivational": y_train.motivational, 
                         "overall": y_train.overall_sentiment},
                    batch_size=32,
                    epochs=2, #50
                    verbose=2
                   )

Epoch 1/2


/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


175/175 - 88s - loss: 5.6610 - sarcasm_loss: 1.1923 - humuor_loss: 1.3002 - offensive_loss: 1.2008 - motivational_loss: 0.6513 - overall_loss: 1.3163 - sarcasm_acc: 0.5046 - sarcasm_f1_score: 0.0000e+00 - humuor_acc: 0.3506 - humuor_f1_score: 0.0000e+00 - offensive_acc: 0.3927 - offensive_f1_score: 0.0000e+00 - motivational_acc: 0.6492 - motivational_f1_score: 0.0000e+00 - overall_acc: 0.4406 - overall_f1_score: 0.0000e+00 - 88s/epoch - 503ms/step
Epoch 2/2
175/175 - 70s - loss: 5.5537 - sarcasm_loss: 1.1688 - humuor_loss: 1.2930 - offensive_loss: 1.1698 - motivational_loss: 0.6484 - overall_loss: 1.2737 - sarcasm_acc: 0.5099 - sarcasm_f1_score: 0.0000e+00 - humuor_acc: 0.3413 - humuor_f1_score: 0.0000e+00 - offensive_acc: 0.3943 - offensive_f1_score: 0.0000e+00 - motivational_acc: 0.6500 - motivational_f1_score: 0.0000e+00 - overall_acc: 0.4412 - overall_f1_score: 0.0000e+00 - 70s/epoch - 399ms/step


In [47]:
pd.DataFrame(history.history)

,loss,sarcasm_loss,humuor_loss,offensive_loss,motivational_loss,overall_loss,sarcasm_acc,sarcasm_f1_score,humuor_acc,humuor_f1_score,offensive_acc,offensive_f1_score,motivational_acc,motivational_f1_score,overall_acc,overall_f1_score
0,5.660958,1.192320,1.300235,1.200764,0.651291,1.316350,0.504566,0.0,0.350582,0.0,0.392659,0.0,0.649239,0.0,0.440645,0.0
1,5.553705,1.168827,1.293023,1.169830,0.648353,1.273672,0.509937,0.0,0.341271,0.0,0.394270,0.0,0.649955,0.0,0.441182,0.0


In [48]:
evaluate = model.evaluate(x = {"image_input": X_test, "text": X_text_test},
                    y = {"sarcasm": y_test.sarcasm, 
                         "humuor": y_test.humour, 
                         "offensive": y_test.offensive, 
                         "motivational": y_test.motivational, 
                         "overall": y_test.overall_sentiment},
                    batch_size=32,
                    verbose=2
                   )

/usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


44/44 - 5s - loss: 5.6374 - sarcasm_loss: 1.2073 - humuor_loss: 1.3047 - offensive_loss: 1.1865 - motivational_loss: 0.6553 - overall_loss: 1.2835 - sarcasm_acc: 0.4689 - sarcasm_f1_score: 0.0000e+00 - humuor_acc: 0.3536 - humuor_f1_score: 0.0000e+00 - offensive_acc: 0.3830 - offensive_f1_score: 0.0000e+00 - motivational_acc: 0.6371 - motivational_f1_score: 0.0000e+00 - overall_acc: 0.4703 - overall_f1_score: 0.0000e+00 - 5s/epoch - 106ms/step


In [51]:
tmp_output = model.predict(x = {"image_input": X_test, "text": X_text_test},
                              batch_size=32,
                              verbose=2)

44/44 - 4s - 4s/epoch - 88ms/step


In [62]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None, 1)]          0           []                               
                                                                                                  
 text_vectorization (TextVector  (None, 100)         0           ['text[0][0]']                   
 ization)                                                                                         
                                                                                                  
 embedding (Embedding)          (None, 100, 50)      500000      ['text_vectorization[0][0]']     
                                                                                                  
 dropout (Dropout)              (None, 100, 50)      0           ['embedding[0][0]']          

In [64]:
from keras.models import Model

embedder= Model(inputs=model.input, outputs=model.layers[-7].output)
embedder.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None, 1)]          0           []                               
                                                                                                  
 text_vectorization (TextVector  (None, 100)         0           ['text[0][0]']                   
 ization)                                                                                         
                                                                                                  
 embedding (Embedding)          (None, 100, 50)      500000      ['text_vectorization[0][0]']     
                                                                                                  
 dropout (Dropout)              (None, 100, 50)      0           ['embedding[0][0]']        

In [67]:
embedded_X_train = embedder.predict(x = {"image_input": X_train, "text": X_text_train},
                              batch_size=32,
                              verbose=2)
embedded_X_test = embedder.predict(x = {"image_input": X_test, "text": X_text_test},
                              batch_size=32,
                              verbose=2)
print(embedded_X_train.shape, embedded_X_test.shape)

175/175 - 10s - 10s/epoch - 59ms/step
44/44 - 2s - 2s/epoch - 53ms/step
(5585, 2560) (1397, 2560)


In [70]:
y_train

,humour,sarcasm,offensive,motivational,overall_sentiment
5751,1,2,1,0,2
608,1,1,1,0,4
3178,2,1,0,0,3
1409,0,0,0,0,3
1571,1,1,0,0,3
...,...,...,...,...,...
6490,2,2,1,0,3
3540,0,1,1,0,3
1133,1,1,0,0,2
5946,0,1,2,1,3


In [73]:
y_train.iloc[:, 0].values

array([1, 1, 2, ..., 1, 0, 2])

In [75]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.svm import SVC
clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
clf.fit(embedded_X_train, y_train.iloc[:, 0].values)
y_pred = clf.predict(embedded_X_test)
print(classification_report(y_test.iloc[:, 0].values, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       334
           1       0.35      1.00      0.52       494
           2       0.50      0.00      0.00       429
           3       0.00      0.00      0.00       140

    accuracy                           0.35      1397
   macro avg       0.21      0.25      0.13      1397
weighted avg       0.28      0.35      0.19      1397



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [76]:
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.ensemble import GradientBoostingClassifier
clf = make_pipeline(StandardScaler(), GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0))
clf.fit(embedded_X_train, y_train.iloc[:, 0].values)
y_pred = clf.predict(embedded_X_test)
print(classification_report(y_test.iloc[:, 0].values, y_pred))

              precision    recall  f1-score   support

           0       0.29      0.08      0.13       334
           1       0.33      0.50      0.40       494
           2       0.30      0.40      0.34       429
           3       0.00      0.00      0.00       140

    accuracy                           0.32      1397
   macro avg       0.23      0.24      0.22      1397
weighted avg       0.28      0.32      0.28      1397

